# Pink Floyd Lyrics

In [2]:
import numpy as np
import pandas as pd

In [11]:
file = "../raw_data/pink_floyd_lyrics.csv"
pink_df = pd.read_csv(file, decimal=",")

In [12]:
pink_df

,album,song_title,year,lyrics
0,The Piper at the Gates of Dawn,Astronomy Domine,1967-08-05,"""Moon in both [houses]...""...Scorpio, [Arabian..."
1,The Piper at the Gates of Dawn,Lucifer Sam,1967-08-05,"Lucifer Sam, siam cat\nAlways sitting by your ..."
2,The Piper at the Gates of Dawn,Matilda Mother,1967-08-05,There was a king who ruled the land\nHis Majes...
3,The Piper at the Gates of Dawn,Flaming,1967-08-05,Alone in the clouds all blue\nLying on an eide...
4,The Piper at the Gates of Dawn,Pow R. Toc H.,1967-08-05,TCH TCH\nAHH (AHH)\nTCH TCH\nAHH AHH\nDoi doi\...
...,...,...,...,...
158,The Endless River,Surfacing,2014-11-07,NaN
159,The Endless River,Louder than Words,2014-10-14,We bitch and we fight\nDiss each other on sigh...
160,The Endless River,TBS9,2014-11-07,NaN
161,The Endless River,TBS14,2014-11-07,NaN


In [13]:
pink_df = pink_df.replace(r'\n',' ', regex=True) 

In [16]:
pink_df['year'] = pd.to_datetime(pink_df.year)

In [20]:
pink_df.dropna(subset = ["lyrics"], inplace=True)


In [29]:
pink_df.groupby('album').count()

,song_title,year,lyrics
album,,,
A Momentary Lapse of Reason,9,9,9
A Saucerful of Secrets,6,6,6
Animals,5,5,5
Atom Heart Mother,5,5,5
Meddle,6,6,6
More (Original Film Soundtrack),7,7,7
Obscured By Clouds,6,6,6
The Dark Side of the Moon,9,9,9
The Division Bell,9,9,9
